# <center>Real-time emotion detection</center>

Authors : DOUET Marie, GRINDEL Brice, MARTIN Lucas, SOUVANNAVONG Elise

## Global imports

In [11]:
import numpy as np
import os
import soundfile
import librosa
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm_notebook
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

## Global functions

In [6]:
def extract_feature(file_name, mfcc=True, chroma=True, mel=True):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")

        sample_rate = sound_file.samplerate

        if chroma:
            stft = np.abs(librosa.stft(X))
        result=np.array([])

        # MFCC Criterias (short-term power spectrum of a sound)
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))

        # Pertains to the 12 diffrent pitch classes
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        
        # MEL Spectrogram Frequency
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
    return result

## Benchmark of datasets

### RAVDESS

In [7]:
emotions = {
    '01': "neutral", 
    '02': "calm", 
    '03': "happy", 
    '04': "sad", 
    '05': "angry", 
    '06': "fearful", 
    '07': "disgust", 
    '08': "surprised"
}

# Removed 'calm' emotion
observed_emotions = ['neutral', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']

In [8]:
def load_data(folder_path, emotions, excluded_files=[], test_size=0.2):
    x,y = [],[]
    files = os.listdir(folder_path)
    
    # iterations with the progress bar
    for i in tqdm_notebook(range(len(files)-len(excluded_files))):
        file = files[i]
        
        if file not in excluded_files:
            file_name = os.path.basename(file)
            emotion = emotions[file_name.split("-")[2]]

            if emotion not in observed_emotions:
                continue

            feature = extract_feature(folder_path + "/" + file)

            x.append(feature)
            y.append(emotion)
    return np.array(x), np.array(y)

In [9]:
ravdess_folder = "./data_samples/RAVDESS"
excluded_files = ["README.md","03-01-06-01-01-02-20.wav", "03-01-08-01-02-02-01.wav", "03-01-03-01-02-01-20.wav"]

x,y = load_data(ravdess_folder, emotions=emotions, excluded_files=excluded_files)
x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle=True, test_size=0.25, random_state=42)

  0%|          | 0/1436 [00:00<?, ?it/s]

## Preprocessing

### MFCC

## Benchmark of models

### 1 - Support Vector Machine (SVM)

In [6]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid = {
    'C': [0.1, 1, 10, 100, 1000],  
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
    'kernel': ['rbf','linear']
}

grid_svm = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose=3)
grid_svm.fit(x_train, y_train) 

print(grid_svm.best_params_)

{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}


In [18]:
# Select best hyper-parameters for SVM model
# C=100, gamma=0.0001, kernel=rbf 
C_opti, gamma_opti, kernel_opti = grid_svm.best_params_['C'], grid_svm.best_params_['gamma'], grid_svm.best_params_['kernel']

svc = svm.SVC(C=C_opti, gamma=gamma_opti, kernel=kernel_opti)
svc.fit(x_train, y_train)

y_pred = svc.predict(x_test)

accuracy = round(accuracy_score(y_true=y_test,y_pred=y_pred)*100,2)
print("Accuracy : ", accuracy,"%")

Accuracy :  62.06 %


### 2 - Multi-layer Perceptron Classifer (MLPC)

In [40]:
param_grid = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

grid_mlpc = GridSearchCV(MLPClassifier(max_iter=100), param_grid, n_jobs=-1, cv=3, verbose=0)
grid_mlpc.fit(x_train, y_train)

print(grid_mlpc.best_params_)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy

[CV 2/3] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=sgd;, score=0.155 total time=   0.4s
[CV 3/3] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=sgd;, score=0.152 total time=   0.3s
[CV 1/3] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=sgd;, score=0.190 total time=   0.9s
[CV 1/3] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=adaptive, solver=sgd;, score=0.129 total time=   1.3s
[CV 2/3] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=adaptive, solver=sgd;, score=0.165 total time=   1.4s
[CV 2/3] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=adam;, score=0.416 total time=   1.5s


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 1/3] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=adam;, score=0.426 total time=   1.7s
[CV 3/3] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=adaptive, solver=sgd;, score=0.158 total time=   1.5s
[CV 3/3] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50, 50), learning_rate=constant, solver=adam;, score=0.429 total time=   1.8s
[CV 2/3] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 100, 50), learning_rate=constant, solver=sgd;, score=0.165 total time=   0.5s


...

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 2/3] END activation=relu, alpha=0.05, hidden_layer_sizes=(100,), learning_rate=constant, solver=adam;, score=0.465 total time=   1.2s
[CV 2/3] END activation=relu, alpha=0.05, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd;, score=0.171 total time=   0.8s
[CV 3/3] END activation=relu, alpha=0.05, hidden_layer_sizes=(100,), learning_rate=constant, solver=sgd;, score=0.155 total time=   1.5s
[CV 3/3] END activation=relu, alpha=0.05, hidden_layer_sizes=(50, 100, 50), learning_rate=adaptive, solver=adam;, score=0.442 total time=   1.7s
[CV 1/3] END activation=relu, alpha=0.05, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd;, score=0.148 total time=   1.0s
[CV 3/3] END activation=relu, alpha=0.05, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd;, score=0.165 total time=   0.8s
[CV 3/3] END activation=relu, alpha=0.05, hidden_layer_sizes=(100,), learning_rate=constant, solver=adam;, score=0.432 total time=   1.3s


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 2/3] END activation=relu, alpha=0.05, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=adam;, score=0.413 total time=   0.6s
[CV 1/3] END activation=relu, alpha=0.05, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=adam;, score=0.452 total time=   0.9s
[CV 3/3] END activation=relu, alpha=0.05, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=adam;, score=0.419 total time=   0.6s
{'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'solver': 'adam'}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [42]:
# Select best hyper-parameters for MLPC model
# activation=tanh, alpha=0.05, hidden_layer_sizes=(100,), learning_rate=constant, solver=adam
activation_opti, alpha_opti, hidden_layer_sizes_opti, learnin_rate_opti, solver_opti = grid_mlpc.best_params_['activation'], grid_mlpc.best_params_['alpha'], grid_mlpc.best_params_['hidden_layer_sizes'], grid_mlpc.best_params_['learning_rate'], grid_mlpc.best_params_['solver']

mlpc = MLPClassifier(activation=activation_opti, solver=solver_opti, alpha=alpha_opti, batch_size=256, epsilon=1e-08, hidden_layer_sizes=hidden_layer_sizes_opti, learning_rate=learnin_rate_opti, max_iter=500)
mlpc.fit(x_train,y_train)

y_pred = mlpc.predict(x_test)

accuracy = round(accuracy_score(y_true=y_test,y_pred=y_pred)*100,2)
print("Accuracy : ", accuracy,"%")

Accuracy :  60.13 %


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


### 3 - Random Forest

In [47]:
param_grid = {
    'max_depth':[3,5,10,None],
    'n_estimators':[10,100,200],
    'max_features':[1,3,5,7],
    'min_samples_leaf':[1,2,3],
    'min_samples_split':[1,2,3]
}

grid_rforest = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=-1, cv=3, verbose=0)
grid_rforest.fit(x_train, y_train)

print(grid_rforest.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
432 fits failed out of a total of 1296.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
432 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 476, in fit
    trees = Parallel(
  File "/usr/local/lib/python3.10/dist-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.10/dist-packages/joblib/parall

{'max_depth': None, 'max_features': 7, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [49]:
# Select best hyper-parameters for Random Forest model
# max_depth=None, max_features=7, min_samples_leaf=1, min_samples_split=2, n_estimators=200
max_depth_opti, max_features_opti, min_samples_leaf_opti, min_samples_split_opti, n_estimators_opti = grid_rforest.best_params_['max_depth'], grid_rforest.best_params_['max_features'], grid_rforest.best_params_['min_samples_leaf'], grid_rforest.best_params_['min_samples_split'], grid_rforest.best_params_['n_estimators']

rforest = RandomForestClassifier(max_depth=max_depth_opti, max_features=max_features_opti, min_samples_leaf=min_samples_leaf_opti, min_samples_split=min_samples_split_opti, n_estimators=n_estimators_opti)
rforest.fit(x_train, y_train)

y_pred = rforest.predict(x_test)

accuracy = round(accuracy_score(y_true=y_test,y_pred=y_pred)*100,2)
print("Accuracy : ", accuracy,"%")

Accuracy :  54.66 %


### Linear Prediction Cepstral Coefficient (LPCC)

## CNN

In [36]:
labels = []
for emotion in y : 
    l = np.argmax(emotion)
    labels.append(l)

In [59]:
# Étiquetage des émotions
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
y = to_categorical(y, num_classes=len(label_encoder.classes_))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#x_train.shape = (930,180)

# Création du modèle CNN
model = models.Sequential()
model.add(layers.Reshape((180, 1), input_shape=(180,)))  # Reshape pour ajouter la dimension du canal
model.add(layers.Conv1D(32, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(128, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(len(label_encoder.classes_), activation='softmax'))

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test))

# Évaluation du modèle
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_acc}')


Epoch 1/20
31/31 [==============================] - 1s 15ms/step - loss: 2.1748 - accuracy: 0.2218 - val_loss: 1.8078 - val_accuracy: 0.1847
Epoch 2/20
31/31 [==============================] - 0s 9ms/step - loss: 1.7362 - accuracy: 0.3226 - val_loss: 1.7420 - val_accuracy: 0.3534
Epoch 3/20
31/31 [==============================] - 0s 10ms/step - loss: 1.5891 - accuracy: 0.3841 - val_loss: 1.5708 - val_accuracy: 0.3976
Epoch 4/20
31/31 [==============================] - 0s 10ms/step - loss: 1.4884 - accuracy: 0.4204 - val_loss: 1.6524 - val_accuracy: 0.3614
Epoch 5/20
31/31 [==============================] - 0s 10ms/step - loss: 1.4192 - accuracy: 0.4657 - val_loss: 1.4237 - val_accuracy: 0.4819
Epoch 6/20
31/31 [==============================] - 0s 10ms/step - loss: 1.3430 - accuracy: 0.5111 - val_loss: 1.4985 - val_accuracy: 0.4458
Epoch 7/20
31/31 [==============================] - 0s 9ms/step - loss: 1.2330 - accuracy: 0.5373 - val_loss: 1.3710 - val_accuracy: 0.5221
Epoch 8/20
31/3

## Results of the benchmark

### Cross validation

In [50]:
from sklearn.model_selection import cross_val_score

svm_scores = cross_val_score(svc, x,y, cv=5)
mlpc_scores = cross_val_score(mlpc, x,y, cv=5)
rforest_scores = cross_val_score(rforest, x,y, cv=5)

names = ["SVM", "MLPC", "Random Forest"]
means = [np.mean(svm_scores), np.mean(mlpc_scores), np.mean(rforest_scores)]
stds = [np.std(svm_scores), np.std(mlpc_scores), np.std(rforest_scores)]
mins = [min(svm_scores), min(mlpc_scores), min(rforest_scores)]
maxs = [max(svm_scores), max(mlpc_scores), max(rforest_scores)]

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

In [51]:
data = {
    'name': names,
    'min': mins,
    'mean': means,
    'max': maxs,
    'var': stds
}

df = pd.DataFrame(data=data)
df

,name,min,mean,max,var
0,SVM,0.552419,0.605966,0.681452,0.045476
1,MLPC,0.629032,0.655904,0.678715,0.015798
2,Random Forest,0.528226,0.560827,0.574297,0.016983
